## Understanding Scorer Metrics

PyRIT has metrics for many scorers checked in. Before diving into how to retrieve or create metrics, it's important to understand what they measure and how scorer identity determines which metrics apply.

### Scorer Identity

Every scorer has a unique **identity hash** computed from its complete configuration:
- Scorer type (e.g., `SelfAskRefusalScorer`)
- System and user prompt templates
- Target model information (endpoint, model name)
- Temperature and other generation parameters
- Any scorer-specific configuration

This means changing *any* of these values creates a new scorer identity. The reason these are variables is because they _might_ change performance—does changing the temperature increase or decrease accuracy? Metrics let you experiment and find out.

Metrics are stored and retrieved by this identity hash, so the same scorer configuration will always get the same cached metrics.

In [1]:
from pyrit.prompt_target import OpenAIChatTarget
from pyrit.score import SelfAskRefusalScorer
from pyrit.setup import IN_MEMORY, initialize_pyrit_async

await initialize_pyrit_async(memory_db_type=IN_MEMORY)

# Create a refusal scorer
refusal_scorer = SelfAskRefusalScorer(chat_target=OpenAIChatTarget())

# View the scorer's full identity - this determines which metrics apply
scorer_identity = refusal_scorer.scorer_identifier
print("Scorer Identity:")
print(f"  Type: {scorer_identity.type}")
print(f"  System Prompt Hash: {scorer_identity.system_prompt_template[:50] if scorer_identity.system_prompt_template else 'None'}...")
print(f"  Target Info: {scorer_identity.target_info}")
print(f"  Identity Hash: {scorer_identity.compute_hash()}")

Found default environment files: ['C:\\Users\\rlundeen\\.pyrit\\.env', 'C:\\Users\\rlundeen\\.pyrit\\.env.local']
Loaded environment file: C:\Users\rlundeen\.pyrit\.env
Loaded environment file: C:\Users\rlundeen\.pyrit\.env.local
Scorer Identity:
  Type: SelfAskRefusalScorer
  System Prompt Hash: You are an evaluator tasked with determining if th...
  Target Info: {'__type__': 'OpenAIChatTarget', 'model_name': 'gpt-40'}
  Identity Hash: d9007ba254383ed04e7240f5bcf6ac6d5e34d79dd7653a800d2f0ed4a26a84ba


### Objective Metrics

Objective scorers produce true/false outputs, usually whether an objective was met (e.g., did this response have instructions on "how to make a Molotov cocktail?"). We evaluate them using standard classification metrics by comparing model predictions against human-labeled ground truth.

- **Accuracy**: Proportion of predictions matching human labels. Simple but can be misleading with imbalanced datasets.
- **Precision**: Of all "true" predictions, how many were correct? High precision = few false positives.
- **Recall**: Of all actual "true" cases, how many did we catch? High recall = few false negatives.
- **F1 Score**: Harmonic mean of precision and recall. Balances both concerns.
- **Accuracy Standard Error**: Statistical uncertainty in accuracy estimate, useful for confidence intervals.

**Which metric matters most?**
- If false positives are costly (e.g., flagging safe content as harmful) → prioritize **precision**
- If false negatives are costly (e.g., missing actual jailbreaks) → prioritize **recall**
- For balanced scenarios → use **F1 score**

### Harm Metrics

Harm scorers produce float scores (0.0-1.0) representing severity. Since these are continuous values, we use different metrics that capture how close the model's scores are to human judgments.

**Error Metrics:**
- **Mean Absolute Error (MAE)**: Average absolute difference between model and human scores. An MAE of 0.15 means the model is off by ~15% on average.
- **MAE Standard Error**: Uncertainty in the MAE estimate.

**Statistical Significance:**
- **t-statistic**: From a one-sample t-test. Positive = model scores higher than humans; negative = lower.
- **p-value**: If < 0.05, the difference between model and human scores is statistically significant (not due to chance).

**Inter-Rater Reliability (Krippendorff's Alpha):**
Measures agreement between evaluators, ranging from -1.0 to 1.0:
- **1.0**: Perfect agreement
- **0.8+**: Strong agreement
- **0.6-0.8**: Moderate agreement
- **< 0.6**: Weak agreement (scorer may need tuning)

Three alpha values are reported:
- **`krippendorff_alpha_humans`**: Agreement among human evaluators (baseline quality of labels)
- **`krippendorff_alpha_model`**: Agreement across multiple scoring trials (model consistency)
- **`krippendorff_alpha_combined`**: Overall agreement between humans and model

## Retrieving Scorer Metrics

Once scorer metrics are calculated with `evaluate_async()`, they're saved to JSONL registry files and can be retrieved without re-running the evaluation. The PyRIT team has pre-computed metrics for common scorer configurations which you can access immediately.

### Retrieving Cached Metrics for a Scorer

Use `get_scorer_metrics()` on any scorer instance to retrieve cached results matching its identity:

In [ ]:
import os
from pyrit.score import TrueFalseInverterScorer, SelfAskRefusalScorer, ConsoleScorerPrinter
from pyrit.prompt_target import OpenAIChatTarget

# This is a simple objective scorer that only detects whether the response was a refusal
objective_scorer = TrueFalseInverterScorer(
        scorer=SelfAskRefusalScorer(
            chat_target=OpenAIChatTarget(
                endpoint=os.environ.get("AZURE_OPENAI_GPT4O_ENDPOINT"),
                api_key=os.environ.get("AZURE_OPENAI_GPT4O_KEY"),
                model_name=os.environ.get("AZURE_OPENAI_GPT4O_MODEL"),
        )
    )
)

# Retrieve pre-computed metrics (from PyRIT team's evaluation runs)
# using the scorer's identity hash
cached_metrics = objective_scorer.get_scorer_metrics()

if cached_metrics:
    print("Evaluation Metrics:")
    print(f"  Dataset Name: {cached_metrics.dataset_name}")
    print(f"  Dataset Version: {cached_metrics.dataset_version}")
    print(f"  F1 Score: {cached_metrics.f1_score}")
    print(f"  Accuracy: {cached_metrics.accuracy}")
    print(f"  Precision: {cached_metrics.precision}")
    print(f"  Recall: {cached_metrics.recall}")
    print(f"  Avg Score time: {cached_metrics.average_score_time_seconds} seconds")
else:
    print("No cached metrics found for this scorer configuration.")

Evaluation Metrics:
  Dataset Name: objective/objective_achieved_metrics.jsonl
  Dataset Version: 1.0
  F1 Score: 0.8395061728395061
  Accuracy: 0.8243243243243243
  Precision: 0.7727272727272727
  Recall: 0.918918918918919
  Avg Score time: 0.43015325405403027 seconds


Harm scorer metrics are retrieved similarly:

In [3]:
import os
from pyrit.score import SelfAskLikertScorer, LikertScalePaths
from pyrit.prompt_target import OpenAIChatTarget

harm_scorer = SelfAskLikertScorer(
        chat_target=OpenAIChatTarget(
            endpoint=os.environ.get("AZURE_OPENAI_GPT4O_ENDPOINT"),
            api_key=os.environ.get("AZURE_OPENAI_GPT4O_KEY"),
            model_name=os.environ.get("AZURE_OPENAI_GPT4O_MODEL"),
        ),
        likert_scale=LikertScalePaths.EXPLOITS_SCALE,
    )

# Retrieve pre-computed metrics using the scorer's identity hash
harm_metrics = harm_scorer.get_scorer_metrics()

if harm_metrics:
    print("Evaluation Metrics:")
    print(f"  Dataset Name: {harm_metrics.dataset_name}")
    print(f"  Dataset Version: {harm_metrics.dataset_version}")
    print(f"  Krippendorff Alpha: {harm_metrics.krippendorff_alpha_combined:.3f}")
    print(f"  P value: {harm_metrics.p_value:.4f}")
    print(f"  Avg Score time: {harm_metrics.average_score_time_seconds} seconds")

else:
    print("No cached metrics found for this scorer configuration.")

Evaluation Metrics:
  Dataset Name: harm/exploits_metrics.jsonl
  Dataset Version: 1.0
  Krippendorff Alpha: 0.458
  P value: 0.0002
  Avg Score time: 0.7639685599999557 seconds


### Comparing All Scorer Configurations

The evaluation registry stores metrics for all tested scorer configurations. You can load all entries to compare which configurations perform best.

Use `get_all_objective_metrics()` or `get_all_harm_metrics()` to load evaluation results. These return `ScorerMetricsWithIdentity` objects with clean attribute access to both the scorer identity and its metrics.

In [4]:
from pyrit.score import get_all_objective_metrics

# Load all objective scorer metrics - returns ScorerMetricsWithIdentity[ObjectiveScorerMetrics]
all_scorers = get_all_objective_metrics()

print(f"Found {len(all_scorers)} scorer configurations in the registry\n")

# Sort by F1 score - type checker knows entry.metrics is ObjectiveScorerMetrics
sorted_by_f1 = sorted(all_scorers, key=lambda x: x.metrics.f1_score, reverse=True)

print("Top 5 configurations by F1 Score:")
print("-" * 80)
for i, entry in enumerate(sorted_by_f1[:5], 1):
    m = entry.metrics  # ObjectiveScorerMetrics - no cast needed!
    print(f"\n{i}. F1={m.f1_score:.3f}  Accuracy={m.accuracy:.2%}  Precision={m.precision:.3f}  Recall={m.recall:.3f}")
    print(f"   Scorer Type: {entry.scorer_identifier.to_compact_dict()}")

# Show best by each metric
print("\n" + "=" * 80)
print(f"Best Accuracy:  {max(all_scorers, key=lambda x: x.metrics.accuracy).metrics.accuracy:.2%}")
print(f"Best Precision: {max(all_scorers, key=lambda x: x.metrics.precision).metrics.precision:.3f}")
print(f"Best Recall:    {max(all_scorers, key=lambda x: x.metrics.recall).metrics.recall:.3f}")
print(f"Fastest:        {min(all_scorers, key=lambda x: x.metrics.average_score_time_seconds).metrics.average_score_time_seconds:.3f} seconds")
print(f"Slowest:        {max(all_scorers, key=lambda x: x.metrics.average_score_time_seconds).metrics.average_score_time_seconds:.3f} seconds")


Found 7 scorer configurations in the registry

Top 5 configurations by F1 Score:
--------------------------------------------------------------------------------

1. F1=0.840  Accuracy=82.43%  Precision=0.773  Recall=0.919
   Scorer Type: {'__type__': 'TrueFalseInverterScorer', 'system_prompt_template': None, 'user_prompt_template': None, 'sub_identifier': [{'__type__': 'SelfAskRefusalScorer', 'system_prompt_template': 'sha256:8e8ddfa0aa41578a', 'user_prompt_template': None, 'sub_identifier': None, 'target_info': {'__type__': 'OpenAIChatTarget', 'model_name': 'gpt-4o-japan-nilfilter'}, 'score_aggregator': 'OR_', 'scorer_specific_params': None}], 'target_info': None, 'score_aggregator': 'OR_', 'scorer_specific_params': None, 'hash': '1db406024b6cb2bb32a89dc7208f765668fc0596a872cab50747a58da2fa8aa0'}

2. F1=0.826  Accuracy=79.73%  Precision=0.724  Recall=0.959
   Scorer Type: {'__type__': 'TrueFalseInverterScorer', 'system_prompt_template': None, 'user_prompt_template': None, 'sub_identi

Similarly, you can look at the best harm scorers for a given category:

In [5]:
from pyrit.score import get_all_harm_metrics

# Load all harm scorer metrics for a specific category
all_harm_scorers = get_all_harm_metrics(harm_category="violence")

print(f"Found {len(all_harm_scorers)} harm scorer configurations for violence\n")

# Sort by Krippendorff's alpha (higher is better)
sorted_by_alpha = sorted(all_harm_scorers, key=lambda x: x.metrics.krippendorff_alpha_combined, reverse=True)

print("Top configurations by Krippendorff's Alpha:")
print("-" * 80)
for i, entry in enumerate(sorted_by_alpha[:5], 1):
    m = entry.metrics
    print(f"\n{i}. K Alpha={m.krippendorff_alpha_combined:.3f}  MAE={m.mean_absolute_error:.3f}  p-value={m.p_value:.4f}  Avg Score time: {m.average_score_time_seconds} seconds")
    print(f"   Scorer Type: {entry.scorer_identifier.to_compact_dict()}")

Found 2 harm scorer configurations for violence

Top configurations by Krippendorff's Alpha:
--------------------------------------------------------------------------------

1. K Alpha=0.868  MAE=0.171  p-value=0.3052  Avg Score time: 0.8147548685714442 seconds
   Scorer Type: {'__type__': 'SelfAskLikertScorer', 'system_prompt_template': 'sha256:c8e2c8367b8f24a3', 'user_prompt_template': None, 'sub_identifier': None, 'target_info': {'__type__': 'OpenAIChatTarget', 'model_name': 'gpt-4o-japan-nilfilter'}, 'score_aggregator': None, 'scorer_specific_params': None, 'hash': '1969e7b7f18b6d7016d5c7184042c58474ee3ba4033d2e5ec9ea8e0d100b2a1b'}

2. K Alpha=0.850  MAE=0.213  p-value=0.4873  Avg Score time: 0.11533883523800233 seconds
   Scorer Type: {'__type__': 'AzureContentFilterScorer', 'system_prompt_template': None, 'user_prompt_template': None, 'sub_identifier': None, 'target_info': None, 'score_aggregator': None, 'scorer_specific_params': {'score_categories': ['Violence']}, 'hash': '3f87

## Creating Scorer Metrics

This section covers how to create new metrics by running evaluations against human-labeled datasets.

### Caching and Skip Logic

When you call `evaluate_async()` on a scorer, the evaluation framework follows a smart caching strategy to avoid redundant work. It checks the metrics registry (a JSONL file) for an existing entry matching the scorer's identity hash. The decision to skip or run evaluation depends on:

1. **No existing entry**: Run the full evaluation
2. **Dataset version changed**: Re-run and replace the old entry (assumes newer dataset is authoritative)
3. **Same version, sufficient trials**: Skip if existing `num_scorer_trials >= requested` (existing metrics are good enough)
4. **Same version, fewer trials**: Re-run with more trials and replace (higher fidelity needed)

During evaluation, the scorer processes each entry from human-labeled CSV dataset(s). For each `assistant_response` in the CSV, the scorer generates predictions which are compared against the `human_score` column(s). For objective scorers, this produces accuracy/precision/recall/F1 metrics. For harm scorers, it calculates MAE, t-statistics, and Krippendorff's alpha.

Setting `add_to_evaluation_results=False` bypasses caching entirely—always running fresh evaluations without reading from or writing to the registry. This is useful for testing custom configurations without polluting the official metrics.

### Running an Objective Evaluation

Call `evaluate_async()` on any scorer instance. The scorer's identity (including system prompt, model, temperature) determines which cached results apply.

In [6]:
from typing import cast
from pyrit.prompt_target import OpenAIChatTarget
from pyrit.score import SelfAskRefusalScorer, ScorerEvalDatasetFiles, ObjectiveScorerMetrics, RegistryUpdateBehavior

# Create a refusal scorer - uses the chat target to determine if responses are refusals
refusal_scorer = SelfAskRefusalScorer(chat_target=OpenAIChatTarget())

# REAL usage would simply be:
# metrics = await refusal_scorer.evaluate_async()

# For demonstration, use a smaller evaluation file (normally you'd use the full dataset)
# The evaluation_file_mapping tells the evaluator which human-labeled CSV files to use
refusal_scorer.evaluation_file_mapping = ScorerEvalDatasetFiles(
    human_labeled_datasets_files=["refusal_scorer/mini_refusal.csv"],
    result_file="refusal_scorer/test_refusal_metrics.jsonl",
)

# Run evaluation with:
# - num_scorer_trials=1: Score each response once (use 3+ for production to measure variance)
# - RegistryUpdateBehavior.NEVER_UPDATE: Don't save to the official registry (for testing only)
metrics = await refusal_scorer.evaluate_async(num_scorer_trials=1, update_registry_behavior=RegistryUpdateBehavior.NEVER_UPDATE)

if metrics:
    objective_metrics = cast(ObjectiveScorerMetrics, metrics)
    print(f"Evaluation Metrics created {objective_metrics.dataset_name}")
else:
    raise RuntimeError("Evaluation failed, no metrics returned")

Evaluation Metrics created refusal_scorer/test_refusal_metrics.jsonl


### Running a Harm Evaluation

In [8]:
from typing import cast
from pyrit.score import SelfAskLikertScorer, LikertScalePaths
from pyrit.score.scorer_evaluation.scorer_evaluator import ScorerEvalDatasetFiles
from pyrit.score.scorer_evaluation.scorer_metrics import HarmScorerMetrics

# Create a harm scorer using the hate speech Likert scale
likert_scorer = SelfAskLikertScorer(
    chat_target=OpenAIChatTarget(),
    likert_scale=LikertScalePaths.HATE_SPEECH_SCALE
)

# Configure evaluation to use a small sample dataset
likert_scorer.evaluation_file_mapping = ScorerEvalDatasetFiles(
    human_labeled_datasets_files=["harm/mini_hate_speech.csv"],
    result_file="harm/test_hate_speech_metrics.jsonl",
    harm_category="hate_speech",  # Required for harm evaluations
)

# This can be called without parameters to update the registry
metrics = await likert_scorer.evaluate_async(num_scorer_trials=1, update_registry_behavior=RegistryUpdateBehavior.NEVER_UPDATE)

if metrics:
    harm_metrics = cast(HarmScorerMetrics, metrics)
    print(f"Dataset: {harm_metrics.dataset_name} created")
else:
    raise RuntimeError("Evaluation failed, no metrics returned")

Dataset: harm/test_hate_speech_metrics.jsonl created


### Custom Evaluation Datasets

You can evaluate scorers against your own human-labeled datasets using `ScorerEvalDatasetFiles`:

```python
# Configure custom datasets
scorer.evaluation_file_mapping = ScorerEvalDatasetFiles(
    # Glob patterns relative to SCORER_EVALS_PATH
    human_labeled_datasets_files=["my_datasets/*.csv"],
    # Where to store results
    result_file="my_datasets/evaluation_metrics.jsonl",
    # Required for harm scorers, ignored for objective scorers
    harm_category="violence",
)
```

### CSV Human Evaluation Files

Many human scored dataset csv files are available in the `pyrit/datasets/scorer_evals/` directory. These include datasets for refusal detection, hate speech, violence, and other harm categories. You can reference these as templates for creating your own evaluation datasets.

**Required columns:**
- `assistant_response`: The text to score
- `objective`: Required for **objective scorers** - the scoring question (e.g., "Did the response refuse?")
- `harm_category`: Required for **harm scorers** - the harm category (e.g., "hate_speech", "violence")
- `human_score` or `human_score_1`, `human_score_2`, etc.: Ground truth labels from human raters
  - For objective scorers: 0 or 1 (converted to bool)
  - For harm scorers: 0.0-1.0 float values
- `data_type`: Type of content (defaults to "text")